In [74]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
import pandas as pd

import os
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark
# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"
# Start a SparkSession
import findspark
findspark.init()

# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Energy_Cancer").getOrCreate()

# Read in data
from pyspark import SparkFiles

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [84]:
import requests
url = "https://raw.githubusercontent.com/fraserclaire/Group3_Project4/main/Artien/Machine%20Learning%20Dataset-Cancer%26Energy.csv"
response = requests.get(url)
# with open("Machine Learning Dataset-Cancer&Energy.csv", "wb") as f:
#     f.write(response.content)

spark.sparkContext.addFile(url)
cancer_energy = spark.read.csv(SparkFiles.get('Machine Learning Dataset-Cancer&Energy.csv'),inferSchema=True,header=True)

# Convert Spark DataFrame to pandas DataFrame
cancer_energy_df = cancer_energy.toPandas()

# Show DataFrame
cancer_energy_df.head()

,_c0,State Abbreviation,2011(C),2012(C),2013(C),2014(C),2015(C),2016(C),2017(C),2018(C),...,2019_Population,2020_LungBronchus,2020_LungBronchus_perCapita,2020_Population,2021 Lung & Bronchus,2021_Population,2021_LungBronchus_perCapita,2011-2020 allsites/capita AVG,2011-2020 lungBronchus/capita AVG,Lung/Bronchus_Classifer
0,0,AK,15481,15521,14819,18225,19511,16613,16376,17325,...,731545,400,0.000546,732964.0,370,734923,0.000503,0.004628,0.000591,0
1,1,AL,651032,547004,565051,575912,494311,410168,378890,377152,...,4903185,4230,0.000841,5031864.0,4520,5050380,0.000895,0.005554,0.000867,1
2,2,AR,306119,296732,327099,339214,226889,246437,267591,304145,...,3017804,2760,0.000916,3014348.0,2970,3028443,0.000981,0.005473,0.000898,1
3,3,AZ,459909,420570,454865,447849,385822,323878,334512,331492,...,7278717,4200,0.000584,7186683.0,4550,7272487,0.000626,0.004939,0.000595,1
4,4,CA,55264,43832,38151,39486,30967,32077,33663,33256,...,39512223,18040,0.000457,39503200.0,17760,39145060,0.000454,0.004456,0.000473,0


In [85]:
cancer_energy_df.drop(columns='_c0', inplace=True)
cancer_energy_df

,State Abbreviation,2011(C),2012(C),2013(C),2014(C),2015(C),2016(C),2017(C),2018(C),2019(C),...,2019_Population,2020_LungBronchus,2020_LungBronchus_perCapita,2020_Population,2021 Lung & Bronchus,2021_Population,2021_LungBronchus_perCapita,2011-2020 allsites/capita AVG,2011-2020 lungBronchus/capita AVG,Lung/Bronchus_Classifer
0,AK,15481,15521,14819,18225,19511,16613,16376,17325,17620,...,731545,400,0.000546,732964.0,370,734923,0.000503,0.004628,0.000591,0
1,AL,651032,547004,565051,575912,494311,410168,378890,377152,317180,...,4903185,4230,0.000841,5031864.0,4520,5050380,0.000895,0.005554,0.000867,1
2,AR,306119,296732,327099,339214,226889,246437,267591,304145,239798,...,3017804,2760,0.000916,3014348.0,2970,3028443,0.000981,0.005473,0.000898,1
3,AZ,459909,420570,454865,447849,385822,323878,334512,331492,257727,...,7278717,4200,0.000584,7186683.0,4550,7272487,0.000626,0.004939,0.000595,1
4,CA,55264,43832,38151,39486,30967,32077,33663,33256,30886,...,39512223,18040,0.000457,39503200.0,17760,39145060,0.000454,0.004456,0.000473,0
5,CO,368871,370085,363532,350526,340107,321476,315835,284474,273338,...,5758736,2550,0.000441,5785219.0,2570,5811596,0.000442,0.004483,0.000457,0
6,CT,6081,9290,7679,9097,6536,2340,2507,4023,867,...,3565287,2650,0.000741,3577586.0,2750,3603691,0.000763,0.006009,0.000754,1
7,DC,48,77,4,48,44,34,30,37,5,...,705749,300,0.000447,670839.0,360,669037,0.000538,0.004538,0.000485,1
8,DE,17881,17384,18254,10238,7149,8199,4804,4296,2179,...,973764,890,0.000897,991862.0,910,1004881,0.000906,0.005954,0.000878,1
9,FL,552730,482984,505155,557882,466530,426188,407538,327807,233542,...,21477737,18150,0.000841,21591299.0,18470,21830708,0.000846,0.006080,0.000881,0


In [72]:
# Split our preprocessed data into our features and target arrays
y = energy_cancer.LungBronchus_Classifer.values
X = energy_cancer.drop(columns="LungBronchus_Classifer").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

AttributeError: 'DataFrame' object has no attribute 'LungBronchus_Classifer'

In [50]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [51]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=200, activation='relu', input_dim=100))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 200)               20200     
                                                                 
 dense_7 (Dense)             (None, 50)                10050     
                                                                 
 dense_8 (Dense)             (None, 1)                 51        
                                                                 
Total params: 30301 (118.36 KB)
Trainable params: 30301 (118.36 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [52]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [59]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_test, y_test))

Epoch 1/50
2/2 [==============================] - 0s 194ms/step - loss: 0.0605 - accuracy: 1.0000 - val_loss: 219520.2812 - val_accuracy: 0.6154
Epoch 2/50
2/2 [==============================] - 0s 116ms/step - loss: 0.0578 - accuracy: 1.0000 - val_loss: 232215.1562 - val_accuracy: 0.6154
Epoch 3/50
2/2 [==============================] - 0s 51ms/step - loss: 0.0582 - accuracy: 1.0000 - val_loss: 238898.1406 - val_accuracy: 0.6154
Epoch 4/50
2/2 [==============================] - 0s 71ms/step - loss: 0.0582 - accuracy: 1.0000 - val_loss: 239718.5000 - val_accuracy: 0.6154
Epoch 5/50
2/2 [==============================] - 0s 73ms/step - loss: 0.0551 - accuracy: 1.0000 - val_loss: 238875.5781 - val_accuracy: 0.6154
Epoch 6/50
2/2 [==============================] - 0s 64ms/step - loss: 0.0576 - accuracy: 1.0000 - val_loss: 244986.5156 - val_accuracy: 0.6154
Epoch 7/50
2/2 [==============================] - 0s 66ms/step - loss: 0.0597 - accuracy: 1.0000 - val_loss: 264510.6250 - val_accurac

In [60]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1/1 - 0s - loss: 1.0069 - accuracy: 0.6154 - 22ms/epoch - 22ms/step
Loss: 1.0068869590759277, Accuracy: 0.6153846383094788
